In [2]:
!mkdir github & cd github & git clone https://github.com/Farama-Foundation/ViZDoom.git

A subdirectory or file github already exists.
fatal: destination path 'ViZDoom' already exists and is not an empty directory.


In [1]:
pwd

'C:\\Users\\Asif'

In [2]:
# Import vizdoom for game env
from vizdoom import * 
from vizdoom import DoomGame
# Import random for action sampling
import random
# Import time for sleeping
import time 
# Import numpy for identity matrix
import numpy as np

In [3]:
game = DoomGame()
game.load_config('C:/Users/Asif/github/ViZDoom/scenarios/defend_the_center.cfg')
game.init()

In [4]:
# This is the set of actions we can take in the environment
actions = np.identity(3, dtype=np.uint8)

In [5]:
# for i in range(100):
#     game.make_action(random.choice(actions))

In [6]:
# Loop through episodes 
episodes = 1 
for episode in range(episodes): 
    # Create a new episode or game 
    game.new_episode()
    # Check the game isn't done 
    while not game.is_episode_finished(): 
        # Get the game state 
        state = game.get_state()
        # Get the game image 
        img = state.screen_buffer
        #print(img.shape) state as an image (3,240,320)
        # Get the game variables - ammo
        info = state.game_variables
        # Take an action
        reward = game.make_action(random.choice(actions),4)
        # Print reward 
        print('reward:', reward) 
        time.sleep(0.02)
    print('Result:', game.get_total_reward())
    time.sleep(2)

reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 1.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 1.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
rewa

In [29]:
## Now converting to GYM ENVIROMENT

In [7]:
!pip install gym

In [8]:
!pip install opencv-python

In [7]:
# Import environment base class from OpenAI Gym
#from gym import Env
# Import gym spaces 
import gymnasium as gym
from gymnasium import spaces
# Import opencv 
import cv2
import matplotlib.pyplot as plt

In [10]:
pip install gymnasium

Note: you may need to restart the kernel to use updated packages.


In [67]:
class VizDoomGym(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 30}

    def __init__(self, render_mode=None):
        super().__init__()
        self.render_mode = render_mode

        self.game = DoomGame()
        self.game.load_config('C:/Users/Asif/github/ViZDoom/scenarios/defend_the_center.cfg')

        if render_mode == "human":
            self.game.set_window_visible(True)
        else:
            self.game.set_window_visible(False)

        self.game.init()

        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(low=0, high=255, shape=(100, 160, 1), dtype=np.uint8)
        self.state = None

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.game.new_episode()
        raw = self.game.get_state().screen_buffer
        self.state = self._preprocess(raw)
        return self.state, {}

    def step(self, action):
        actions = np.identity(3, dtype=np.uint8)
        reward = self.game.make_action(actions[action], 4)

        terminated = self.game.is_episode_finished()
        truncated = False

        if not terminated:
            raw = self.game.get_state().screen_buffer
            self.state = self._preprocess(raw)
            ammo = self.game.get_state().game_variables[0]
        else:
            self.state = np.zeros(self.observation_space.shape, dtype=np.uint8)
            ammo = 0

        return self.state, reward, terminated, truncated, {"ammo": ammo}

    def render(self):
        if self.render_mode == "rgb_array":
        # Return the latest frame as RGB
            img = np.squeeze(self.state, axis=-1)  # (H, W)
            return cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
            
        elif self.render_mode == "human":
        # VizDoom already shows the game window in human mode
            pass
        
    def close(self):
        self.game.close()

    def _preprocess(self, observation):
        img = np.moveaxis(observation, 0, -1)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (160, 100), interpolation=cv2.INTER_AREA)
        return resized[..., np.newaxis].astype(np.uint8)

In [47]:
env = VizDoomGym(render_mode="human")

In [45]:
state = env.reset()

In [50]:
env.close()

In [48]:
from stable_baselines3.common import env_checker

In [49]:
env_checker.check_env(env)

In [64]:
#!pip install matplotlib

   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.1 MB 2.8 MB/s eta 0:00:03
   ----------- ---------------------------- 2.4/8.1 MB 6.7 MB/s eta 0:00:01
   ------------------ --------------------- 3.7/8.1 MB 6.6 MB/s eta 0:00:01
   ------------------------------- -------- 6.3/8.1 MB 8.2 MB/s eta 0:00:01
   ------------------------------- -------- 6.3/8.1 MB 8.2 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 7.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ------------------------------------- -- 2.1/2.2 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 9.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ------------- -------------------------- 2.4/7.0 MB 11.2 MB/s eta 0:00:01
   ------------------------- -------------- 4.5/7.0 MB 11.2 MB/s eta 0:00:01
   ---------------------------

In [ ]:
plt.imshow(cv2.cvtColor(state, cv2.COLOR_BGR2RGB))

In [80]:
## Setting up callback

In [84]:
!pip install -v torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Using pip 25.1 from C:\Users\Asif\anaconda3\envs\vizdoom-env\Lib\site-packages\pip (python 3.12)
Looking in indexes: https://download.pytorch.org/whl/cu118


In [83]:
!pip install stable-baselines3[extra]

   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.5 MB ? eta -:--:--
   ------ --------------------------------- 0.5/3.5 MB 1.2 MB/s eta 0:00:03
   ------ --------------------------------- 0.5/3.5 MB 1.2 MB/s eta 0:00:03
   ------ --------------------------------- 0.5/3.5 MB 1.2 MB/s eta 0:00:03
   --------- ------------------------------ 0.8/3.5 MB 610.3 kB/s eta 0:00:05
   --------- ------------------------------ 0.8/3.5 MB 610.3 kB/s eta 0:00:05
   --------- ------------------------------ 0.8/3.5 MB 610.3 kB/s eta 0:00:05
   ------------ --------------------------- 1.0/3.5 MB 572.0 kB/s eta 0:00:05
   ------------ --------------------------- 1.0/3.5 MB 572.0 kB/s eta 0:00:05
   ------------ --------------------------- 1.0/3.5 MB 572.0 kB/s eta 0:00:05
   ------------ --------------------------- 1.0/3.5 MB 572.0 kB/s eta 0:00:05
   ------------ --------------------------- 1.0/3.5 MB 572.0 kB/s eta 0:00:05
   -------

In [51]:
# Import os for file nav
import os 
# Import callback class from sb3
from stable_baselines3.common.callbacks import BaseCallback

In [56]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [54]:
CHECKPOINT_DIR = './train/train_center'
LOG_DIR = './logs/log_center'

In [55]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [21]:
#Training

In [20]:
from stable_baselines3 import PPO

In [23]:
env = VizDoomGym()

In [25]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.0001, n_steps=4096)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [ ]:
model.learn(total_timesteps=100000, callback=callback)

Logging to ./logs/log_center\PPO_1


In [27]:
#test

In [21]:
from stable_baselines3.common.evaluation import evaluate_policy

In [89]:
model = PPO.load('C:/Users/Asif/train/train_center/best_model_100000')

In [124]:
env = VizDoomGym(render_mode="human")

In [73]:
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=10)

In [74]:
mean_reward

np.float64(19.5)

In [125]:
for episode in range(10):
    obs, _ = env.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(obs)
        obs, reward, terminated, truncated, info = env.step(action)
        time.sleep(0.025)
        done = terminated or truncated
        total_reward += reward
    print(f"Total Reward for episode {episode} is {total_reward}")

env.close()

Total Reward for episode 0 is 9.0
Total Reward for episode 1 is 17.0
Total Reward for episode 2 is 16.0
Total Reward for episode 3 is 20.0
Total Reward for episode 4 is 16.0
Total Reward for episode 5 is 16.0


ViZDoomUnexpectedExitException: Controlled ViZDoom instance exited unexpectedly.

In [2]:
pip install moviepy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
import numpy as np

# Input paths
video1_path = r"C:\Users\Asif\Downloads\RL_VID.mp4"
video2_path = r"C:\Users\Asif\Downloads\this_2.mp4"

# Output path
output_path = r"C:\Users\Asif\Downloads\merged_side_by_side.mp4"

# Open video files
cap1 = cv2.VideoCapture(video1_path)
cap2 = cv2.VideoCapture(video2_path)

# Get properties from first video
fps = int(cap1.get(cv2.CAP_PROP_FPS))
width1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Get properties from second video
fps2 = int(cap2.get(cv2.CAP_PROP_FPS))
width2 = int(cap2.get(cv2.CAP_PROP_FRAME_WIDTH))
height2 = int(cap2.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Use min FPS to avoid speed mismatch
fps_out = min(fps, fps2)

# Resize second video to match first video's height
new_width2 = int(width2 * (height1 / height2))
size = (width1 + new_width2, height1)

# Create video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps_out, size)

while True:
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    if not ret1 or not ret2:
        break  # stop when any video ends

    frame2 = cv2.resize(frame2, (new_width2, height1))

    # Combine horizontally
    combined = np.hstack((frame1, frame2))

    out.write(combined)

cap1.release()
cap2.release()
out.release()

print(f"✅ Merged video saved to: {output_path}")


✅ Merged video saved to: C:\Users\Asif\Downloads\merged_side_by_side.mp4
